# Problema 1: Resolución de estado desfavorable (Clasificación binaria)

#### Carga de librerías


In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from keras.wrappers.scikit_learn import KerasClassifier

#### Configuración de figuras

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.rc('figure', autolayout=True)

plt.rc('font', size=10)
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=24)
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('legend', fontsize=22.5)
plt.rc('figure', titlesize=50)
plt.rc('animation', html='html5')

#### Carga de datos

In [ ]:
df = pd.read_csv('1_train.csv', index_col=0)

df_test = pd.read_csv('1_test.csv', index_col=0)

df.head()

#### Procesamiento de datos para el ingreso a la red neuronal

In [ ]:
X = df.copy()
y = X.pop('natural')

X_test = df_test.copy()
y_test = X_test.pop('natural')

features_num = ['time', 'blocked_anchors']
features_cat = ['station_num', 'status', 'wday', 'mday', 'month']

transformer_num = make_pipeline(
    SimpleImputer(strategy="median"), # manejo de valores faltantes
    StandardScaler(),                 # estandarización
)
transformer_cat = make_pipeline(
    SimpleImputer(strategy="most_frequent", fill_value="NA"), # manejo de valores faltantes
    OneHotEncoder(handle_unknown='ignore', drop=None, sparse = True),
)
preprocessor = make_column_transformer(
    (transformer_num, features_num),
    (transformer_cat, features_cat),
)

# Partición aleatoria en entrenamiento (75%) y validación (25%)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.75, random_state=22, shuffle=False)

print('shape = ', X_train.shape)
print('valid = ', X_valid.shape)

X_train = preprocessor.fit_transform(X_train)
X_valid = preprocessor.transform(X_valid)
X_test = preprocessor.transform(X_test)

print('shape = ', X_train.shape)
print('valid = ', X_valid.shape)

input_shape = [X_train.shape[1]]
print("Input shape: {}".format(input_shape))

X_train = X_train.toarray()
X_valid = X_valid.toarray()
X_test = X_test.toarray()

#### Grid Search

In [ ]:
# Función con hiperparámetros a optimizar
def create_model(optimizer='adam', activation = 'relu', hidden_layers=1, hidden_size=8):
  # Inicializar el constructor
    model = keras.Sequential()
      # Capa de entrada
    model.add(layers.BatchNormalization(input_shape = input_shape))

    if activation == 'relu':
      initializer = 'he_normal'
    else:
      initializer = 'glorot_normal'

    for i in range(hidden_layers):
        # Capa oculta
        model.add(layers.Dense(hidden_size, activation=activation, kernel_initializer=initializer))

      # Capa de salida
    model.add(layers.Dense(1, activation='sigmoid'))
      # Compilar el modelo
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Modelo a utilizar como input de la función GridSearchCV
modelCV = KerasClassifier(build_fn=create_model, verbose=0)

In [ ]:
# Diccionario con grilla de valores de hiperparámetros
param_grid = dict(
    hidden_layers = [1, 2], 
    hidden_size = [8, 16, 32, 64], 
    activation = ['relu', 'tanh'], 
    optimizer = ['sgd', 'rmsprop', 'adam'], 
    batch_size = [10**2], 
    epochs = [10]
)

# Implementación de grid search
grid = GridSearchCV(estimator=modelCV, param_grid=param_grid, scoring='accuracy')
grid_result = grid.fit(X_train, y_train)

In [ ]:
# Imprimir resultados
print('Best accuracy:', grid_result.best_score_)
grid_result.best_params_

#### Entrenamiento de red

In [ ]:
# Definir el modelo
model = keras.Sequential([
    layers.BatchNormalization(input_shape = input_shape),
    
    layers.Dense(64, activation = 'tanh', kernel_initializer='glorot_normal'),
    layers.BatchNormalization(),
    
    layers.Dense(64, activation = 'tanh', kernel_initializer='glorot_normal'),
    layers.BatchNormalization(),
    
    layers.Dense(1, activation = 'sigmoid')
])

# Compilar el modelo
model.compile(optimizer = 'rmsprop', 
       loss = keras.losses.BinaryCrossentropy(), 
       metrics = [keras.metrics.BinaryAccuracy()])

In [ ]:
# Detención temprana
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

# Entrenar el modelo
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=10**2,
    epochs=50,
    callbacks=[early_stopping],
    verbose = 0
)

#### Guardar modelo

In [ ]:
model.save("model1.h5")

#### Cargar modelo

In [ ]:
from keras.models import load_model
model = load_model("model1.h5")

#### Curva ROC

In [ ]:
y_train_pred = model.predict(X_valid)
fpr, tpr, thresholds = roc_curve(y_valid, y_train_pred, drop_intermediate=True)
roc_auc = auc(fpr, tpr)

print(len(fpr))

plt.figure(figsize=(8, 6))
lw = 5
plt.plot(fpr, tpr, color="darkorange", lw=lw, 
         label="Curva ROC (área = %0.2f)" % roc_auc,)
plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("1 - Especificidad")
plt.ylabel("Sensibilidad")
#plt.title("Curva ROC")
plt.legend(loc="lower right")
plt.show()

#### Punto de corte óptimo

In [ ]:
ROCdf = pd.DataFrame({'fpr': fpr, 'tpr': tpr, 'cutoff': thresholds})

# Criterios para seleccionar un punto de corte
ROCdf['dist'] = ROCdf.fpr**2 + (1-ROCdf.tpr)**2
ROCdf['youden'] =  ROCdf.tpr - ROCdf.fpr
ROCdf['diag'] = (ROCdf.tpr + ROCdf.fpr - 1)**2

# Punto de corte según cada criterio
print(ROCdf[ROCdf.dist == ROCdf.dist.min()])
print(ROCdf[ROCdf.youden == ROCdf.youden.max()])
print(ROCdf[ROCdf.diag == ROCdf.diag.min()])

#### Respuesta predicha

In [ ]:
cutoff = float(ROCdf[ROCdf.youden == ROCdf.youden.max()].cutoff)

y_pred = model.predict(X_test)
y_pred = [1 if y > cutoff else 0 for y in y_pred]

# Precisión
accuracy_score(y_test, y_pred)

#### Matriz de confusión

In [ ]:
cf_matrix = [[0.18, 0.13], [0.06, 0.63]]
annot_extra = [['(n=2968)', '(n=2068)'], ['(n=940)', '(n=10380)']]
vmin = np.min(cf_matrix)
vmax = np.max(cf_matrix)

fig, ax = plt.subplots(figsize=(9,6))

sns.heatmap(cf_matrix, annot=cf_matrix, annot_kws={'size': 40, 'va': 'bottom'}, fmt='.0%', cmap='Blues', vmin=0, vmax=1, cbar=False)
sns.heatmap(cf_matrix, annot=annot_extra, annot_kws={'size': 40, 'va': 'top'}, fmt="", cmap='Blues', vmin=0, vmax=1, cbar=False)

plt.yticks(rotation=0) 

ax.set_ylabel('Resolución observada')
ax.set_xticklabels(['No natural', 'Natural'])
ax.set_yticklabels(['No natural', 'Natural'])
ax.set_xlabel('Resolución predicha')